In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/ShaneKoNaung/llama2-train-from-scratch.git


Cloning into 'llama2-train-from-scratch'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 75 (delta 27), reused 63 (delta 17), pack-reused 0
Receiving objects: 100% (75/75), 1.87 MiB | 4.87 MiB/s, done.
Resolving deltas: 100% (27/27), done.


In [3]:
%cd llama2-train-from-scratch

/content/llama2-train-from-scratch


## Download dataset

### using script

In [4]:
!cp -rv  /content/drive/MyDrive/data.zip /content/llama2-train-from-scratch/data/

'/content/drive/MyDrive/data.zip' -> '/content/llama2-train-from-scratch/data/data.zip'


In [5]:
!python prepare_thai_data.py download

data/data.zip already exists, skipping download...
Unpacking data/data.zip...
Archive:  data/data.zip
   creating: data/prachathai67k/data/
  inflating: data/prachathai67k/data/train.jsonl  
  inflating: data/prachathai67k/data/test.jsonl  
  inflating: data/prachathai67k/data/valid.jsonl  
Download done.
Number of shards: 3
Example story:
{'url': 'https://prachatai.com/print/62490', 'date': '2015-11-17 18:14', 'title': 'แฮคเกอร์ Anonymous ลั่นทำสงครามไซเบอร์ครั้งใหญ่สุดกับกลุ่ม IS', 'body_text': '17 พ.ย. 2558 Blognone [1] รายงานว่า กลุ่มแฮคเกอร์ Anonymous ประกาศสงครามไซเบอร์กับกลุ่มหัวรุนแรงหลังจากกลุ่ม IS ออกมาประกาศว่าเป็นผู้อยู่เบื้องหลังการโจมตีกรุงปารีสในคืนวันศุกร์ที่ผ่านมา\n\n\nภาพในคลิปใน YouTube โฆษกของกลุ่มแฮคเกอร์สวมหน้ากากที่เป็นสัญลักษณ์ของกลุ่มได้ออกมาอ่านแถลงเป็นภาษาฝรั่งเศส มีใจความว่า จากการโจมตีของกลุ่ม IS ในกรุงปารีส กลุ่ม Anonymous ทั่วโลกจะตามล่ากลุ่ม IS เหมือนที่เคยทำตอนที่มีการโจมตีสำนักพิมพ์ Charlie Hebdo และครั้งนี้จะเป็นปฏิบัติการโจมตีครั้งใหญ่ที่สุดของกลุ่ม 

In [6]:
!python prepare_thai_data.py prepare

Number of train samples : 61100
Number of test samples : 6789
Train sample : 
	วุฒิสภาจี้หาทางออกเหมืองโปแตช ประชาไท9 ก.พ. 2549 เมื่อวันที่ 8 ก.พ. เวลาประมาณ 14.00 น. คณะกรรมาธิการสิ่งแวดล้อมวุฒิสภาได้จัดเวทีประชุมติดตามความคืบหน้าโครงการเหมืองแร่โปแตช จ. อุดรธานี เนื่องจากเห็นว่ามีการผลักดันโครงการอย่างเร่งด่วนโดยไม่คำนึงถึงการมีส่วนร่วมของประชาชนในพื้นที่ ณ ห้องประชุมกรรมาธิการหมายเลข 306 อาคารรัฐสภาตามข้อเรียกร้องของกลุ่มอนุรักษ์สิ่งแวดล้อมอุดรธานี โดยได้เชิญหน่วยงานที่เกี่ยวข้องได้แก่ กรมอุตสาหกรรมพื้นฐานและการเหมืองแร่ สำนักงานนโยบายและแผนทรัพยากรธรรมชาติสิ่งแวดล้อม กลุ่มอนุรักษ์สิ่งแวดล้อมอุดรธานี เข้ามาชี้แจงให้ข้อมูล      นายสุรพงษ์ เชียงทอง เจ้าหน้าที่กรมอุตสาหกรรมพื้นฐานและการเหมืองแร่ (กพร.) รายงานต่อที่ประชุมว่าขณะนี้บริษัทได้ยื่นขอประธานบัตรทำเหมืองแล้วอยู่ระหว่างการขึ้นรูปแผนที่พื้นที่ทำเหมืองใต้ดิน ในขณะนี้ทางบริษัทต้องทำการรังวัดปักหมดเขตที่ตั้งโรงแยกแร่ หรือเหมืองแร่บนดินและขึ้นรูปแผนที่เพื่อจะได้ติดประกาศในท้องถิ่นก่อนที่อธิบดีกพร.จะรับรอง แต่อย่างไรก็ตามขณะนี้บริษัย

In [7]:
!python prepare_thai_data.py pretokenize

Saved data/prachathai67k/test.bin, average seqlen: 5961.4124
Saved data/prachathai67k/train.bin, average seqlen: 5866.3181


### on notebook

In [ ]:
import os
import requests
import glob
import json

from tqdm import tqdm

In [ ]:
DATA_CACHE_DIR = "data"
data_dir = os.path.join(DATA_CACHE_DIR, 'prachathai67k')

In [ ]:
os.makedirs(DATA_CACHE_DIR, exist_ok=True)


In [ ]:
def download_file(url: str, fname: str, chunk_size=1024):

    resp = requests.get(url, stream=True)
    total = int(resp.headers.get("content-length", 0))
    with open(fname, "wb") as file, tqdm(
            desc=fname,
            total=total,
            unit="iB",
            unit_scale=True,
            unit_divisor=1024,
    ) as bar:
        for data in resp.iter_content(chunk_size=chunk_size):
            size = file.write(data)
            bar.update(size)

def download():
    os.makedirs(DATA_CACHE_DIR, exist_ok=True)

    data_url = "https://archive.org/download/prachathai67k/data.zip"
    data_filename = os.path.join(DATA_CACHE_DIR, "data.zip")
    if not os.path.exists(data_filename):
        print(f"Downloading {data_url} to {data_filename}...")
        download_file(data_url, data_filename)
    else:
        print(f"{data_filename} already exists, skipping download...")

    data_dir = os.path.join(DATA_CACHE_DIR, "prachathai67k")
    if not os.path.exists(data_dir):
        os.makedirs(data_dir, exist_ok=True)
        print(f"Unpacking {data_filename}...")
        os.system(f"unzip {data_filename} -d {data_dir}")
        os.system(f"mv {data_dir}/data/* {data_dir}")
        os.system(f"rm -rf {data_dir}/data/")
    else:
        print(f"{data_dir} already exists, skipping unpacking...")

    shard_filenames = sorted(glob.glob(os.path.join(data_dir, "*.jsonl")))
    data = []
    with open(shard_filenames[0], "r") as f:
        for line in f:
            data.append(json.loads(line))

    print("Download done.")
    print(f"Number of shards: {len(shard_filenames)}")
    print(f"Example story:\n{data[0]}")

In [ ]:
def prepare():
    '''
    combine train.jsonl and valid.jsonl and convert all three jsonl to

    <title> <bodytext> <topic> ...
    '''
    data_dir = os.path.join(DATA_CACHE_DIR, "prachathai67k")
    shard_filenames = sorted(glob.glob(os.path.join(data_dir, "*.jsonl")))

    txt_filename = os.path.join(data_dir, 'train.txt')


    train = []
    test = []

    if not os.path.exists(txt_filename):

        for shard in shard_filenames:
            if 'train' in shard or 'valid' in shard:
                with open(shard, "r") as f:
                    for line in f:
                        data = json.loads(line)
                        text = [v for k,v in data.items() if k in ['title', 'body_text']]
                        text = [i.replace('\n', ' ') for i in text]
                        text.extend([k for k, v in data.items() if v == 1])
                        text = ' '.join(text)
                        text.replace('\n', ' ')
                        train.append(text)
            else:
                with open(shard, "r") as f:
                    for line in f:
                        data = json.loads(line)
                        text = [v for k,v in data.items() if k in ['title', 'body_text']]
                        text = [i.replace('\n', ' ') for i in text]
                        text.extend([k for k, v in data.items() if v == 1])
                        text = ' '.join(text)
                        text.replace('\n', ' ')
                        test.append(text)

        train_txt_filename = os.path.join(data_dir, 'train.txt')
        test_txt_filename  = os.path.join(data_dir, 'test.txt')
        with open(train_txt_filename, 'w') as f:
            for i in train:
                f.write(i + '\n')

        with open(test_txt_filename, 'w') as f:
            for i in test:
                f.write(i + '\n')
    else:
        train_txt_filename = os.path.join(data_dir, 'train.txt')
        test_txt_filename  = os.path.join(data_dir, 'test.txt')
        print(f"{txt_filename} already exists, skipping preparing...")

    train = []
    test = []
    with open(train_txt_filename, 'r') as f:
        for i in f:
            train.append(i.strip())

    with open(test_txt_filename, 'r') as f:
        for i in f:
            test.append(i.strip())


    print(f"Number of train samples : {len(train)}")
    print(f"Number of test samples : {len(test)}")

    print(f"Train sample : \n\t{train[0]}\n")
    print(f"Test sample : \n\t{test[0]}\n")


In [ ]:
download()

data/data.zip: 100%|██████████| 242M/242M [07:17<00:00, 582kiB/s]


Unpacking data/data.zip...
Download done.
Number of shards: 3
Example story:
{'url': 'https://prachatai.com/print/62490', 'date': '2015-11-17 18:14', 'title': 'แฮคเกอร์ Anonymous ลั่นทำสงครามไซเบอร์ครั้งใหญ่สุดกับกลุ่ม IS', 'body_text': '17 พ.ย. 2558 Blognone [1] รายงานว่า กลุ่มแฮคเกอร์ Anonymous ประกาศสงครามไซเบอร์กับกลุ่มหัวรุนแรงหลังจากกลุ่ม IS ออกมาประกาศว่าเป็นผู้อยู่เบื้องหลังการโจมตีกรุงปารีสในคืนวันศุกร์ที่ผ่านมา\n\n\nภาพในคลิปใน YouTube โฆษกของกลุ่มแฮคเกอร์สวมหน้ากากที่เป็นสัญลักษณ์ของกลุ่มได้ออกมาอ่านแถลงเป็นภาษาฝรั่งเศส มีใจความว่า จากการโจมตีของกลุ่ม IS ในกรุงปารีส กลุ่ม Anonymous ทั่วโลกจะตามล่ากลุ่ม IS เหมือนที่เคยทำตอนที่มีการโจมตีสำนักพิมพ์ Charlie Hebdo และครั้งนี้จะเป็นปฏิบัติการโจมตีครั้งใหญ่ที่สุดของกลุ่ม Anonymous เลย นอกจากนี้กลุ่ม Anonymous ยังแสดงความเสียใจต่อครอบครัวผู้สูญเสียในเหตุการณ์ครั้งนี้\nกลุ่ม Anonymous เคยประกาศสงครามกับกลุ่ม IS หลังจากการโจมตีสำนักพิมพ์ Charlie Hebdo ที่ฝรั่งเศสเมื่อต้นปีที่ผ่านมา ซึ่งครั้งนั้นกลุ่ม Anonymous อ้างว่าได้ระงับบัญชีผู้ใ

In [ ]:
!cp -rv /content/llama2-train-from-scratch/data/data.zip /content/drive/MyDrive/

'/content/llama2-train-from-scratch/data/data.zip' -> '/content/drive/MyDrive/data.zip'


In [ ]:
prepare()

Number of train samples : 61100
Number of test samples : 6789
Train sample : 
	วุฒิสภาจี้หาทางออกเหมืองโปแตช ประชาไท9 ก.พ. 2549 เมื่อวันที่ 8 ก.พ. เวลาประมาณ 14.00 น. คณะกรรมาธิการสิ่งแวดล้อมวุฒิสภาได้จัดเวทีประชุมติดตามความคืบหน้าโครงการเหมืองแร่โปแตช จ. อุดรธานี เนื่องจากเห็นว่ามีการผลักดันโครงการอย่างเร่งด่วนโดยไม่คำนึงถึงการมีส่วนร่วมของประชาชนในพื้นที่ ณ ห้องประชุมกรรมาธิการหมายเลข 306 อาคารรัฐสภาตามข้อเรียกร้องของกลุ่มอนุรักษ์สิ่งแวดล้อมอุดรธานี โดยได้เชิญหน่วยงานที่เกี่ยวข้องได้แก่ กรมอุตสาหกรรมพื้นฐานและการเหมืองแร่ สำนักงานนโยบายและแผนทรัพยากรธรรมชาติสิ่งแวดล้อม กลุ่มอนุรักษ์สิ่งแวดล้อมอุดรธานี เข้ามาชี้แจงให้ข้อมูล      นายสุรพงษ์ เชียงทอง เจ้าหน้าที่กรมอุตสาหกรรมพื้นฐานและการเหมืองแร่ (กพร.) รายงานต่อที่ประชุมว่าขณะนี้บริษัทได้ยื่นขอประธานบัตรทำเหมืองแล้วอยู่ระหว่างการขึ้นรูปแผนที่พื้นที่ทำเหมืองใต้ดิน ในขณะนี้ทางบริษัทต้องทำการรังวัดปักหมดเขตที่ตั้งโรงแยกแร่ หรือเหมืองแร่บนดินและขึ้นรูปแผนที่เพื่อจะได้ติดประกาศในท้องถิ่นก่อนที่อธิบดีกพร.จะรับรอง แต่อย่างไรก็ตามขณะนี้บริษัย

## Tokenization

In [ ]:
from typing import List

from sentencepiece import SentencePieceProcessor

In [ ]:
TOKENIZER_MODEL = "tokenizer.model"

In [ ]:
class Tokenizer:
    def __init__(self, tokenizer_model=None):
        model_path = tokenizer_model if tokenizer_model else TOKENIZER_MODEL
        assert os.path.isfile(model_path), model_path
        self.sp_model = SentencePieceProcessor(model_file=model_path)
        self.model_path = model_path

        # BOS / EOS token IDs
        self.n_words: int = self.sp_model.vocab_size()
        self.bos_id: int = self.sp_model.bos_id()
        self.eos_id: int = self.sp_model.eos_id()
        self.pad_id: int = self.sp_model.pad_id()
        #print(f"#words: {self.n_words} - BOS ID: {self.bos_id} - EOS ID: {self.eos_id}")
        assert self.sp_model.vocab_size() == self.sp_model.get_piece_size()

    def encode(self, s: str, bos: bool, eos: bool) -> List[int]:
        assert type(s) is str
        t = self.sp_model.encode(s)
        if bos:
            t = [self.bos_id] + t
        if eos:
            t = t + [self.eos_id]
        return t

    def decode(self, t: List[int]) -> str:
        return self.sp_model.decode(t)


In [ ]:
t = Tokenizer()
print(f"Encode : {t.encode('วุฒิสภาจี้หาทางออกเหมืองโปแตช', 1, 1)}")
print(f"Decode : {t.decode(t.encode('วุฒิสภาจี้หาทางออกเหมืองโปแตช',1, 1))}")

Encode : [1, 29871, 30492, 30803, 227, 187, 149, 30507, 30547, 31070, 30289, 30991, 30691, 30652, 30663, 30289, 30595, 30289, 30398, 30351, 30351, 30425, 30401, 30663, 30501, 31422, 30351, 30398, 31364, 31010, 31073, 30618, 30913, 2]
Decode : วุฒิสภาจี้หาทางออกเหมืองโปแตช


### Tokenize the dataset using llama2 tokenizer

In [ ]:
import numpy as np


In [ ]:
def pretokenize():
    data_dir = os.path.join(DATA_CACHE_DIR, "prachathai67k")
    enc = Tokenizer()

    filenames = sorted(glob.glob(os.path.join(data_dir, "*.txt")))


    for filename in filenames:
        txt_basename = os.path.basename(filename)
        bin_basename = txt_basename.replace('.txt', '.bin')
        tokenized_filename = os.path.join(data_dir, bin_basename)

        if not os.path.exists(tokenized_filename):

            all_tokens = []
            with open(filename, "r") as f:
                for text in f:
                    text = text.strip()
                    if text:
                        tokens = enc.encode(text, bos=True, eos=True)
                        all_tokens.extend(tokens)

            all_tokens = np.array(all_tokens, dtype=np.uint16)



            with open(tokenized_filename, "wb") as f:
                f.write(all_tokens.tobytes())

            avg_seq_len = all_tokens.size / ((all_tokens == 1).sum())
            print(f"Saved {tokenized_filename}, average seqlen: {avg_seq_len:.4f}")
        else:
            print(f"{tokenized_filename} exists,  pretokenization is already done for {txt_basename}")


In [ ]:
pretokenize()

Saved data/prachathai67k/test.bin, average seqlen: 5961.4124
Saved data/prachathai67k/train.bin, average seqlen: 5866.3181


## Batch Iteration

In [ ]:
import random
import glob

import torch
import torch.distributed as dist

from functools import partial

In [ ]:
class PretokDataset(torch.utils.data.IterableDataset):
    def __init__(self, split, max_seq_len, vocab_size):
        super().__init__()
        self.split = split
        self.max_seq_len = max_seq_len
        self.vocab_size = vocab_size

    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        worker_id = worker_info.id if worker_info else 0
        rank = dist.get_rank() if dist.is_initialized() else 0
        seed = 42 + worker_id + 1337 * rank
        rng = random.Random(seed)
        print(f"Created a PretokDataset with rng seed {seed}")

        bin_dir = os.path.join(DATA_CACHE_DIR, "prachathai67k")
        filename = os.path.join(bin_dir, "train.bin") if self.split == "train" else os.path.join(bin_dir, "test.bin")

        assert len(filename)>0, f"No bin files found in {bin_dir}"


        while True:

            m = np.memmap(filename, dtype=np.uint16, mode="r")
            num_batches = len(m) // self.max_seq_len
            num_batches -= 1
            assert num_batches > 0, "this file is way too small? investigatte."
            ixs = list(range(num_batches))
            rng.shuffle(ixs)
            for ix in ixs:
                start = ix * self.max_seq_len
                end = start + self.max_seq_len + 1
                chunk = torch.from_numpy((m[start: end]).astype(np.int64))
                x = chunk[:-1]
                y = chunk[1:]
                yield x,y

In [ ]:
class Task:

    @staticmethod
    def iter_batches(batch_size, device, num_workers=0, **dataset_kwargs):
        ds = PretokDataset(**dataset_kwargs)
        dl = torch.utils.data.DataLoader(
            ds, batch_size=batch_size, pin_memory=False, num_workers=num_workers
        )
        for x, y in dl:
            x = x.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)
            yield x, y

In [ ]:
batch_size = 8
max_seq_len = 128
vocab_size = 32000
split="Train"
device = "cpu"

In [ ]:
iter_batches = partial(
    Task.iter_batches,
    batch_size= batch_size,
    split=split,
    max_seq_len =max_seq_len,
    vocab_size = vocab_size,
    device = device,
    num_workers=0,)

In [ ]:
batch_iter = iter_batches()

In [ ]:
X, y = next(batch_iter)
print(X[:4, :4])
print(y[:4, :4])
X.shape, y.shape

Created a PretokDataset with rng seed 42
tensor([[    1,  1152,  3362,  6669],
        [29901,   437,   366,  1023],
        [ 1244, 12021, 29915, 29881],
        [ 3573, 29915, 29879,  3158]])
tensor([[ 1152,  3362,  6669,   322],
        [  437,   366,  1023,  1073],
        [12021, 29915, 29881,   592],
        [29915, 29879,  3158,  6860]])


(torch.Size([8, 128]), torch.Size([8, 128]))

## Training

In [8]:
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link



### Using script

In [10]:
!python train_thai.py

tokens per iteration will be: 16,384
breaks down as: 4 grad accum steps * 1 processes * 8 batch_size * 512 max seq len
Initializing a new model from scratch
+---------------------------------+------------+
|             Modules             | Parameters |
+---------------------------------+------------+
|      tok_embeddings.weight      |  9216000   |
|   layers.0.attention.wq.weight  |   82944    |
|   layers.0.attention.wk.weight  |   82944    |
|   layers.0.attention.wv.weight  |   82944    |
|   layers.0.attention.wo.weight  |   82944    |
| layers.0.feed_forward.w1.weight |   221184   |
| layers.0.feed_forward.w2.weight |   221184   |
| layers.0.feed_forward.w3.weight |   221184   |
|  layers.0.attention_norm.weight |    288     |
|     layers.0.ffn_norm.weight    |    288     |
|   layers.1.attention.wq.weight  |   82944    |
|   layers.1.attention.wk.weight  |   82944    |
|   layers.1.attention.wv.weight  |   82944    |
|   layers.1.attention.wo.weight  |   82944    |
| layers.1

### On notebook

In [ ]:
import math
import time

from datetime import datetime
from contextlib import nullcontext

from model import Transformer, ModelArgs
from export import model_export

In [ ]:
# -----------------------------------------------------------------------------
# I/O
out_dir = "out_thai"
eval_interval = 2000
log_interval = 1
eval_iters = 1000
eval_only = False  # if True, script exits right after the first eval
always_save_checkpoint = False  # if True, always save a checkpoint after each eval
init_from = "resume"  # 'scratch' or 'resume'
# wandb logging
wandb_log = False  # disabled by default
wandb_project = "llamac"
wandb_run_name = "run" + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
# data
batch_size = 128  # if gradient_accumulation_steps > 1, this is the micro-batch size
max_seq_len = 512
vocab_source = "llama2" # llama2|custom; use Lllama 2 vocab from Meta, or custom trained
vocab_size = 32000 # the Llama 2 tokenizer has 32K tokens
# model
dim = 288
n_layers = 6
n_heads = 6
n_kv_heads = 6
multiple_of = 32
dropout = 0.0
# adamw optimizer
gradient_accumulation_steps = 4  # used to simulate larger batch sizes
learning_rate = 5e-4  # max learning rate
max_iters = 50000  # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0  # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True  # whether to decay the learning rate
warmup_iters = 1000  # how many steps to warm up for
# system
device = "cuda"  # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = "float16"  # float32|bfloat16|float16
compile = True  # use PyTorch 2.0 to compile the model to be faster
# -----------------------------------------------------------------------------

In [ ]:
# fixing some hyperparams to sensible defaults
lr_decay_iters = max_iters  # should be ~= max_iters per Chinchilla
min_lr = 0.0  # minimum learning rate, should be ~= learning_rate/10 per Chinchilla

In [ ]:
master_process = True
seed_offset = 0
ddp_world_size = 1

In [ ]:
tokens_per_iter = gradient_accumulation_steps * ddp_world_size * batch_size * max_seq_len

if master_process:
    print(f"tokens per iteration will be: {tokens_per_iter:,}")
    print(f"breaks down as: {gradient_accumulation_steps} grad accum steps * {ddp_world_size} processes * {batch_size} batch_size * {max_seq_len} max seq len")

if master_process:
    os.makedirs(out_dir, exist_ok=True)

tokens per iteration will be: 262,144
breaks down as: 4 grad accum steps * 1 processes * 128 batch_size * 512 max seq len


In [ ]:
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
device_type = "cuda" if "cuda" in device else "cpu"

ptdtype = {"float32": torch.float32, "bfloat16": torch.bfloat16, "float16": torch.float16}[dtype]

ctx = (
    nullcontext()
    if device_type == "cpu"
    else torch.amp.autocast(device_type=device_type, dtype=ptdtype)
)

In [ ]:
# task-specific setup
iter_batches = partial(
    Task.iter_batches,
    batch_size=batch_size,
    max_seq_len=max_seq_len,
    vocab_size=vocab_size,
    device=device,
    num_workers=0,
)

In [ ]:
# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9

model_args = dict(
    dim=dim,
    n_layers=n_layers,
    n_heads=n_heads,
    n_kv_heads=n_kv_heads,
    vocab_size=vocab_size,
    multiple_of=multiple_of,
    max_seq_len=max_seq_len,
    dropout=dropout,
)

if init_from == "scratch":

    print("Initializing a new model from scratch")
    gptconf = ModelArgs(**model_args)
    model = Transformer(gptconf)

elif init_from == "resume":
    print(f"Resuming training from {out_dir}")
    # resume training from a checkpoint.
    ckpt_path = os.path.join(out_dir, "ckpt.pt")
    checkpoint = torch.load(ckpt_path, map_location=device)
    checkpoint_model_args = checkpoint["model_args"]
    # force these config attributes to be equal otherwise we can't even resume training
    # the rest of the attributes (e.g. dropout) can stay as desired from command line
    for k in ["dim", "n_layers", "n_heads", "n_kv_heads", "vocab_size", "multiple_of", "max_seq_len"]:
        model_args[k] = checkpoint_model_args[k]
    # create the model
    gptconf = ModelArgs(**model_args)
    model = Transformer(gptconf)
    state_dict = checkpoint["model"]
    # fix the keys of the state dictionary :(
    # honestly no idea how checkpoints sometimes get this prefix, have to debug more
    unwanted_prefix = "_orig_mod."
    for k, v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix) :]] = state_dict.pop(k)
    model.load_state_dict(state_dict)
    iter_num = checkpoint["iter_num"]
    best_val_loss = checkpoint["best_val_loss"]
model.to(device)



Resuming training from out_thai


Transformer(
  (tok_embeddings): Embedding(32000, 288)
  (dropout): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0-5): 6 x TransformerBlock(
      (attention): Attention(
        (wq): Linear(in_features=288, out_features=288, bias=False)
        (wk): Linear(in_features=288, out_features=288, bias=False)
        (wv): Linear(in_features=288, out_features=288, bias=False)
        (wo): Linear(in_features=288, out_features=288, bias=False)
        (attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
      (feed_forward): FeedForward(
        (w1): Linear(in_features=288, out_features=768, bias=False)
        (w2): Linear(in_features=768, out_features=288, bias=False)
        (w3): Linear(in_features=288, out_features=768, bias=False)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (attention_norm): RMSNorm()
      (ffn_norm): RMSNorm()
    )
  )
  (norm): RMSNorm()
  (output): Linear(in_features=288, ou

In [ ]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(model)

+---------------------------------+------------+
|             Modules             | Parameters |
+---------------------------------+------------+
|      tok_embeddings.weight      |  9216000   |
|   layers.0.attention.wq.weight  |   82944    |
|   layers.0.attention.wk.weight  |   82944    |
|   layers.0.attention.wv.weight  |   82944    |
|   layers.0.attention.wo.weight  |   82944    |
| layers.0.feed_forward.w1.weight |   221184   |
| layers.0.feed_forward.w2.weight |   221184   |
| layers.0.feed_forward.w3.weight |   221184   |
|  layers.0.attention_norm.weight |    288     |
|     layers.0.ffn_norm.weight    |    288     |
|   layers.1.attention.wq.weight  |   82944    |
|   layers.1.attention.wk.weight  |   82944    |
|   layers.1.attention.wv.weight  |   82944    |
|   layers.1.attention.wo.weight  |   82944    |
| layers.1.feed_forward.w1.weight |   221184   |
| layers.1.feed_forward.w2.weight |   221184   |
| layers.1.feed_forward.w3.weight |   221184   |
|  layers.1.attentio

15191712

In [ ]:
# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == "float16"))

In [ ]:
# optimizer
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)
if init_from == "resume" and "optimizer" in checkpoint:
    optimizer.load_state_dict(checkpoint["optimizer"])
checkpoint = None  # free up memory

num decayed parameter tensors: 43, with 15,187,968 parameters
num non-decayed parameter tensors: 13, with 3,744 parameters
using fused AdamW: True


In [ ]:
# compile the model
if compile:
    print("compiling the model... (takes a ~minute)")
    unoptimized_model = model
    model = torch.compile(model)  # requires PyTorch 2.0


compiling the model... (takes a ~minute)


In [ ]:
# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        batch_iter = iter_batches(split=split)
        losses = torch.zeros(eval_iters)  # keep on CPU
        for k in range(eval_iters):
            X, Y = next(batch_iter)
            with ctx:
                logits = model(X, Y)
                loss = raw_model.last_loss
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))  # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)



In [34]:
# training loop

train_batch_iter = iter_batches(split="train")
X, Y = next(train_batch_iter) # fetch the very first batch
t0 = time.time()
local_iter_num = 0 # number of iterations in the lifetime of this process
raw_model = model

running_mfu = -1.0

while True:
    # determine and set the learning rate for this iteration
    lr = get_lr(iter_num) if decay_lr else learning_rate
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

    if iter_num % eval_interval == 0 and master_process:
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if wandb_log:
            try:
                wandb.log(
                    {
                        "iter": iter_num,
                        "tokens": iter_num * tokens_per_iter,
                        "loss/train": losses["train"],
                        "loss/val": losses["val"],
                        "lr": lr,
                        "mfu": running_mfu * 100,  # convert to percentage
                    }, step = iter_num
                )
            except Exception as e:
                print(f"logging to wandb failed: {e}")
        if losses["val"] < best_val_loss or always_save_checkpoint:
            best_val_loss = losses["val"]
            if iter_num > 0:
                checkpoint = {
                    "model": raw_model.state_dict(),
                    "optimizer": optimizer.state_dict(),
                    "model_args": model_args,
                    "iter_num": iter_num,
                    "best_val_loss": best_val_loss,

                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, "ckpt.pt"))
                model_export(raw_model, os.path.join(out_dir, "model.bin"), version=0)
                !cp -rv out_thai/ /content/drive/MyDrive/
    if iter_num == 0 and eval_only:
        break

    # forward backward update, with optional gradient accumulation to simulate larger batch size
    # and using the GradScaler if data type is float16
    for micro_step in range(gradient_accumulation_steps):

        with ctx:
            logits = model(X, Y)
            loss = raw_model.last_loss
            loss = loss / gradient_accumulation_steps
        # immediately async prefetch next batch while model is doing the forward pass on the GPU
        X, Y = next(train_batch_iter)
        # backward pass, with gradient scaling if training in fp16
        scaler.scale(loss).backward()
    # clip the gradient
    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    # step the optimizer and scaler if training in fp16
    scaler.step(optimizer)
    scaler.update()
    # flush the gradients as soon as we can, no need for this memory anymore
    optimizer.zero_grad(set_to_none=True)

    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0 and master_process:
        # get loss as float, scale up due to the divide above. note: this is a CPU-GPU sync point
        lossf = loss.item() * gradient_accumulation_steps
        if local_iter_num >= 5:  # let the training loop settle a bit
            mfu = raw_model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
            running_mfu = mfu if running_mfu == -1.0 else 0.9 * running_mfu + 0.1 * mfu
        print(
            f"{iter_num} | loss {lossf:.4f} | lr {lr:e} | {dt*1000:.2f}ms | mfu {running_mfu*100:.2f}%"
        )
    iter_num += 1
    local_iter_num += 1

    # termination conditions
    if iter_num > max_iters:
        break

Streaming output truncated to the last 5000 lines.
13542 | loss 1.0021 | lr 4.234364e-04 | 825.13ms | mfu 2.59%
13543 | loss 0.9585 | lr 4.234249e-04 | 821.62ms | mfu 2.59%
13544 | loss 0.9533 | lr 4.234133e-04 | 827.93ms | mfu 2.59%
13545 | loss 0.9643 | lr 4.234018e-04 | 824.86ms | mfu 2.59%
13546 | loss 1.0229 | lr 4.233902e-04 | 827.22ms | mfu 2.59%
13547 | loss 0.9469 | lr 4.233787e-04 | 821.83ms | mfu 2.59%
13548 | loss 1.0061 | lr 4.233671e-04 | 823.23ms | mfu 2.59%
13549 | loss 0.9928 | lr 4.233556e-04 | 823.11ms | mfu 2.59%
13550 | loss 0.9834 | lr 4.233440e-04 | 827.87ms | mfu 2.59%
13551 | loss 0.9516 | lr 4.233325e-04 | 825.21ms | mfu 2.59%
13552 | loss 0.9515 | lr 4.233209e-04 | 822.63ms | mfu 2.59%
13553 | loss 0.9716 | lr 4.233094e-04 | 826.93ms | mfu 2.59%
13554 | loss 0.9476 | lr 4.232978e-04 | 823.72ms | mfu 2.59%
13555 | loss 1.0083 | lr 4.232863e-04 | 825.10ms | mfu 2.59%
13556 | loss 0.9702 | lr 4.232747e-04 | 823.81ms | mfu 2.59%
13557 | loss 0.9486 | lr 4.232632e

KeyboardInterrupt: 

In [ ]:
!cp -rv out_thai/ /content/drive/MyDrive/

'out_thai/' -> '/content/drive/MyDrive/out_thai'
'out_thai/ckpt.pt' -> '/content/drive/MyDrive/out_thai/ckpt.pt'
'out_thai/model.bin' -> '/content/drive/MyDrive/out_thai/model.bin'


Mounted at /content/drive


## Generation

## Run with Llama Meta

## Convert to GGUF

## Run with llama.cpp